In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
import re
import upsetplot
import pyteomics
from commons import data_processing
from commons import common_objects as co 
from pyteomics.mass import Unimod 

alt.data_transformers.disable_max_rows()

In [ ]:
# Define colors used throughout

BROWN = "#604739"
LIGHT_BROWN = "#9a8786"
BLUE_GRAY = "#b3acb9"
LIGHT_BLUE = "#8b97b9"
BLUE = "#637192"
DARK_BLUE = "#3b4459"

colors = [
    BROWN, LIGHT_BROWN, BLUE_GRAY,
    LIGHT_BLUE, BLUE, DARK_BLUE
]

In [ ]:
df = pd.read_parquet(r"./data/DIANN_results/BCaP_6Line_DIANN_report.parquet")

# change column names
df.columns = [c.lower().replace(".", "_") for c in df.columns]

# extract sample name and replicate information from file
def grab_sample_info(s):
    pattern = re.compile(r"BCaP\_([A-Z0-9]+)\-(\d)\_run(\d)")
    match = re.search(pattern, s)    
    return '-'.join(list(match.groups()))
        

info = df.run.map(grab_sample_info)

# split series on '-'
info = info.str.split('-', expand=True)

# rename columns
info.columns = ["sample_name", "bio_rep", "tech_rep"]

# merge into former dataframe
df = df.merge(info, left_index=True, right_index=True)
df.head()

In [ ]:
# create custom sample order for later use
SAMPLES = ["BPH1", "NT1", "T1", "T10", "M1", "MT10"]
BIO_REPS = ["1", "2", "3"]

sample_order = dict(zip(SAMPLES, range(6)))

df["sample_order"] = df.sample_name.map(sample_order)

#### Due to mechanical failure, the final sample (M1-3), was excluded

In [ ]:
# remove sample M1-3
df = df[(df.sample_name!="M1") | (df.bio_rep!="3")]

In [ ]:
# define overall sample order
bcap_order = [
    "BPH1-1", "BPH1-2", "BPH1-3",
    "NT1-1", "NT1-2", "NT1-3",
    "T1-1", "T1-2", "T1-3",
    "T10-1", "T10-2", "T10-3",
    "M1-1", "M1-2", "M1-3",
    "MT10-1", "MT10-2", "MT10-3",
]

In [ ]:
# group on sample and bio rep to find number peptides
grouped = df.groupby(["sample_name", "bio_rep", "modified_sequence"]).mean()
grouped.reset_index("modified_sequence", inplace=True)

# get list of samples
samples = df.sample_name.unique()
reps = df.bio_rep.unique()

# loop and count total/unique
peptide_data = pd.DataFrame()
for s in samples:
    for r in reps:
        if s=="M1" and r=="3":
            # sample thrown out
            continue

        data = grouped.loc[(s, r), :]

        small = pd.DataFrame({
            "sample":f"{s}-{r}",
            "values":[len(data), len(data[data.proteotypic==1])],
            "kind":["Total", "Unique"]
        })
        peptide_data = pd.concat([peptide_data, small]).reset_index(drop=True)

# Get axis from common objects
y_axis = co.alt_axis.copy()
y_axis["tickMinStep"] = 10_000

# make chart
peptide_bars = alt.Chart(
    peptide_data,
).mark_bar(
    width=12,
).encode(
    x=alt.X("kind:N", title="",
                axis=alt.Axis(
                    labelAngle=-45,
                    ticks=False,
                    labels=False)
            ),
    y=alt.Y("values:Q", title="Peptide Count",
                axis=y_axis,
                scale=alt.Scale(
                    domain=[80_000, 130_000])),
    column=alt.Column("sample:N", sort=bcap_order, spacing=10),
    color=alt.Color("kind:N",
                    scale=alt.Scale(
                        domain=["Total", "Unique"],
                        range=[DARK_BLUE, LIGHT_BLUE]
                    ))
).properties(
    width=30,
    height=150
)

In [ ]:
# currently, non-unique proteins are grouped together, need to split
print(f"{df.shape[0]} rows prior to split")

df.loc[:, "protein_ids"] = df.protein_ids.str.split(";")
df = df.explode('protein_ids')

print(f"{df.shape[0]} rows after split")

In [ ]:
# do similar grouping to find total and unique proteins
grouped = df.groupby(["sample_name", "bio_rep", "modified_sequence", "protein_ids"]).mean()
grouped.reset_index(["modified_sequence", "protein_ids"], inplace=True)

# loop and find total/unique
protein_data = pd.DataFrame()
for s in samples:
    for r in reps:
        if s=="M1" and r=="3":
            # sample thrown out
            continue

        data = grouped.loc[(s, r), :]
        
        # total is count of proteins; unique is count of proteins with proteotypic evidence
        small = pd.DataFrame({
            "sample":f"{s}-{r}",
            "values":[len(data.protein_ids.drop_duplicates()), len(data[data.proteotypic==1].protein_ids.drop_duplicates())],
            "kind":["Total", "Unique"]
        })
        protein_data = pd.concat([protein_data, small]).reset_index(drop=True)

y_axis = co.alt_axis.copy()
y_axis["tickMinStep"] = 2_000

protein_bars = alt.Chart(
    protein_data,
).mark_bar(
    width=12,
).encode(
    x=alt.X("kind:N", title="",
                axis=alt.Axis(
                    labelAngle=-45,
                    ticks=False,
                    labels=False)
            ),
    y=alt.Y("values:Q", title="Protein Count",
                axis=y_axis),
    column=alt.Column("sample:N", sort=bcap_order, spacing=10),
    color=alt.Color("kind:N",
                scale=alt.Scale(
                    domain=["Total", "Unique"],
                    range=[DARK_BLUE, LIGHT_BLUE]
                ))
).properties(
    width=30,
    height=150
)

In [ ]:
(peptide_bars & protein_bars).save("./figures/Peptide_Protein_Bars.svg")
peptide_bars & protein_bars

In [ ]:
grouped = df.groupby(["sample_name", "bio_rep", "modified_sequence"], as_index=False).mean()

peptide_counts = pd.DataFrame(
    grouped.modified_sequence.value_counts().value_counts()
    ).sort_index(ascending=False).reset_index()


peptide_counts.columns = ["found_in", "pep_count"]
peptide_counts["missing_in"] = 17 - peptide_counts.found_in
peptide_counts["running_count"] = peptide_counts.pep_count.cumsum()


peptide_steps = alt.Chart(peptide_counts).mark_line(
    interpolate="step-before",
    color=DARK_BLUE,
    # filled=True
).encode(
    x=alt.X("missing_in:Q", title="Missing Values",
            axis=co.alt_axis),
    y=alt.Y("running_count:Q", title="Peptide Count",
            axis=co.alt_axis),
).properties(width=200)

In [ ]:
grouped = df.groupby(["sample_name", "bio_rep", "protein_ids"], as_index=False).mean()

protein_counts = pd.DataFrame(
    grouped.protein_ids.value_counts().value_counts()
    ).sort_index(ascending=False).reset_index()

protein_counts.columns = ["found_in", "pep_count"]
protein_counts["missing_in"] = 17 - protein_counts.found_in
protein_counts["running_count"] = protein_counts.pep_count.cumsum()


x_axis = co.alt_axis.copy()
x_axis["tickMinStep"] = 4


protein_steps = alt.Chart(protein_counts).mark_line(
    interpolate="step-before",
    stroke=LIGHT_BLUE
).encode(
    x=alt.X("missing_in:Q", title="Missing Values",
            axis=x_axis,
            scale=alt.Scale(domain=[0,16])),
    y=alt.Y("running_count:Q", title="Protein Count",
                axis=co.alt_axis),
).properties(width=200)

In [ ]:
(peptide_steps | protein_steps).save("./figures/Peptide_Protein_Steps.svg")
peptide_steps | protein_steps

In [ ]:
# upset plot of protein overlap in all samples

grouped = df.groupby(["sample_order", "sample_name", "bio_rep", "protein_ids"]).mean()
grouped = grouped.sort_index(level=["sample_order", "sample_name", "bio_rep"], ascending=[True, False, True])
grouped.reset_index(["protein_ids", "sample_order"], inplace=True)

protein_overlap = {}
i = 0
for s in SAMPLES:
    overlap = {}
    for r in BIO_REPS:
        if s=="M1" and r=="3":
            # sample thrown out
            continue
        overlap[f"{s}-{r}"] = set(grouped.loc[(s, r), "protein_ids"].unique())
    overlap_data = upsetplot.from_contents(overlap)
    overlap_up = upsetplot.UpSet(overlap_data,
        orientation='vertical', show_counts='%d', 
        show_percentages=True, sort_by='cardinality',  
        sort_categories_by=None, facecolor=colors[i]
        )
    overlap_up.plot()
    plt.gca().set_title(s)
    plt.savefig(f"./figures/Upset_{s}.svg")
    i += 1

In [ ]:

db = Unimod()
PROTON = 1.007277

mod_sequences = df[df.modified_sequence.str.contains(r'UniMod:\d*\)', regex=True)].modified_sequence.drop_duplicates()
mod_string = ' '.join(mod_sequences)
mods = set(re.findall(r'(UniMod:\d*)', mod_string))
print(mods)

mod_lookup = dict() 

for mod in mods:
    mod_id = re.search(r'\d*$', mod).group()
    uni = db.by_id(int(mod_id))

    mod_lookup[mod] = uni['mono_mass']
    print(mod, uni['full_name'], uni['mono_mass'])

def map_mass(sequence, mod_lookup=mod_lookup):
    found_mods = re.findall(r'(UniMod:\d*)', sequence)
    stripped = re.sub(r'\(UniMod:\d*\)', '', sequence)
    mass_change = 0
    if found_mods:
        for mod in found_mods:
            # mod_id = re.search(r'\d+', mod).group()
            mass_change += mod_lookup[mod]
    mass = pyteomics.mass.calculate_mass(stripped) + PROTON
    return mass + mass_change

sequence_all = df.modified_sequence.drop_duplicates()

masses = sequence_all.apply(map_mass)
mass_lookup = dict(zip(sequence_all, masses))

df["mass"] = df.modified_sequence.map(mass_lookup)
df["mz"] = df.mass / df.precursor_charge

mass_frame = df.groupby(["sample_name", "bio_rep", "modified_sequence"]).mean()
mass_frame = mass_frame.reset_index("modified_sequence")
mass_frame = mass_frame.sort_index()

In [ ]:
# get density of m/z values
from scipy import stats

gaussian_mass = pd.DataFrame()
for s in SAMPLES:
    for rep in BIO_REPS:
        if s == "M1" and rep == "3":
            # thrown out
            continue
        data = mass_frame.loc[(s, rep)]
        data = data.drop_duplicates('modified_sequence')
        data = data.sample(frac=0.1)
        values = data.mz.to_numpy()
        gaus = stats.gaussian_kde(values, bw_method=0.05)(values)

        sub = pd.DataFrame({
            'sample':s,
            'ident':f'{s}-{rep}',
            'values':values,
            'gaussian':gaus
        }) 

        gaussian_mass = pd.concat([gaussian_mass, sub])
        gaussian_mass.reset_index(inplace=True, drop=True)

In [ ]:
x_axis = co.alt_axis.copy()
x_axis["tickMinStep"] = 100

base = alt.Chart(gaussian_mass).encode(
    x=alt.X('values:Q', title='m/z',
        axis=x_axis,
        scale=alt.Scale(
            domain=[300,1200])),
    y=alt.Y('gaussian:Q', title='density',
        axis=co.alt_axis),
    color=alt.Color('sample:N',
        scale=alt.Scale(
            domain=SAMPLES,
            range=colors)
        ),
    detail='ident:N',
).properties(width=600,height=150)

alt.layer(base.mark_area(opacity=0.2) + base.mark_line()).facet(row=alt.Row('sample:N', sort=SAMPLES), spacing=50)


In [ ]:
from venn import venn, pseudovenn

overlap = {}
for s in samples:
    s_frame = grouped.loc[(s,), :]
    if s != "M1":
        s_frame_kept = data_processing.get_valid_counts(s_frame, "protein_ids", 3)
    else:
        # continue
        s_frame_kept = data_processing.get_valid_counts(s_frame, "protein_ids", 2)
    overlap[s] = set(s_frame_kept.protein_ids.unique())

pseudovenn(overlap)

In [ ]:
grouped = df.groupby(["sample_name", "bio_rep", "tech_rep", "protein_ids"], as_index=False).mean()


In [ ]:
protein_quant = df[df.proteotypic==1].groupby(["sample_name", "bio_rep", "tech_rep", "protein_ids"]).mean()
protein_quant = protein_quant.reset_index(["tech_rep", "protein_ids"])

data_table = pd.DataFrame()
rep_frame = pd.DataFrame()

for s in SAMPLES:
    for r in BIO_REPS:
        print(s, r)
        run_frame = protein_quant.loc[(s, r), ["protein_ids", "tech_rep", "pg_maxlfq"]]
        run_frame = run_frame[run_frame.pg_maxlfq > 0]
        run_frame = data_processing.get_valid_counts(run_frame, "protein_ids", 2)
        run_frame["log2_int"] = np.log2(run_frame.pg_maxlfq)

        x = run_frame[run_frame.tech_rep=="1"].log2_int
        y = run_frame[run_frame.tech_rep=="2"].log2_int

        if s == "M1" and r == "3":
            x = run_frame[run_frame.tech_rep=="3"].log2_int
            y = run_frame[run_frame.tech_rep=="4"].log2_int

        m, b, r_val, _, _ = stats.linregress(x, y)

        xy = np.vstack([x,y])
        dens = stats.gaussian_kde(xy)(xy)

        data_insert = pd.DataFrame({
            "Sample":f"{s}-{r}",
            "Num Proteins":len(x),
            "Equation":f"y = {m:.3f}x + {b:.3f}",
            "Fit":f"R\u00b2 = {r_val:.3f}"
        }, index=[0])

        rep_insert = pd.DataFrame({
            "sample":s,
            "bio_rep":r,
            "Rep 1, log2(intensity)":x,
            "Rep 2, log2(intensity)":y,
            "density":dens,
            "linex":np.linspace(0, 25, len(x)),
            "liney":m * np.linspace(0, 25, len(x)) + b,
        }).sample(frac=0.33).sort_values("density")

        data_table = pd.concat([data_table, data_insert]).reset_index(drop=True)
        rep_frame = pd.concat([rep_frame, rep_insert]).reset_index(drop=True)

data_table.to_csv("./tables/Intrasample_Reproducibility.csv")

In [ ]:
dots = alt.Chart(rep_frame).mark_circle(
    size=10
).encode(
    x=alt.X("Rep 1, log2(intensity):Q",
        axis=co.alt_axis,
        scale=alt.Scale(domain=[0,25])),
    y=alt.Y("Rep 2, log2(intensity):Q",
        axis=co.alt_axis,
        scale=alt.Scale(domain=[0,25])),
    color=alt.Color("density:Q", legend=None,
        scale=alt.Scale(range=["#BBBBBB", DARK_BLUE])
    )
).properties(
    width=300,
    height=150
)

lines = alt.Chart(rep_frame).mark_line(
    strokeDash=[10, 5],
    color="black"
).encode(
    x=alt.X("linex:Q", title="",
        axis=co.alt_axis),
    y=alt.Y("liney:Q", title="",
        axis=co.alt_axis),
).properties(
    width=300,
    height=150
)

alt.layer(lines, dots).facet(
    row=alt.Row('sample:N', sort=SAMPLES),
    column='bio_rep:Q',
    spacing=40
).resolve_scale(color="independent")